# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('data/song_data')

In [6]:
filepath = song_files[0]
print(len(song_files))


78


In [7]:
df = pd.read_json(filepath, lines = True)
print(df.head())
print([df.values[0][7], df.values[0][8], df.values[0][0], df.values[0][9], df.values[0][5]])
#print(df.values)

   num_songs           artist_id  artist_latitude  artist_longitude  \
0          1  AREDL271187FB40F44              NaN               NaN   

  artist_location   artist_name             song_id  \
0                  Soul Mekanik  SOPEGZN12AB0181B3D   

                              title  duration  year  
0  Get Your Head Stuck On Your Neck  45.66159     0  
['Get Your Head Stuck On Your Neck', 45.66159, 1, 0, 'Soul Mekanik']


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [15]:

df = pd.DataFrame(columns = pd.read_json(song_files[0],lines = True).columns)
for record in song_files:
    data = pd.read_json(record,lines = True)
    df = df.append(data)
    
#    song_data.append([df.values[0][7], df.values[0][8], df.values[0][0], df.values[0][9], df.values[0][5]])
#    artist_data.append([df.values[0][0], df.values[0][4], df.values[0][2], df.values[0][1], df.values[0][3]])
print(len(df))
song_data = pd.DataFrame({'song_id':df['song_id'], 'title':df['title'], 'artist_id':df['artist_id'], 'year':df['year'], 'duration':df['duration']})
artist_data = pd.DataFrame({'artist_id':df['artist_id'], 'name':df['artist_name'], 'location':df['artist_location'], 'latitude':df['artist_latitude'], 'longtitude':df['artist_longitude']})

#print(len(song_data))
#print(len(artist_data))

78


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [17]:

for song in song_data.values:

    cur.execute(song_table_insert, song)
    conn.commit()


['SOPEGZN12AB0181B3D' 'Get Your Head Stuck On Your Neck'
 'AREDL271187FB40F44' 0 45.66159]
['SOTCKKY12AB018A141' 'Sonnerie lalaleulé hi houuu' 'ARGSAFR1269FB35070' 0
 29.54404]
['SOQOTLQ12AB01868D0' 'Clementina Santafè' 'ARGCY1Y1187B9A4FA5' 0
 153.33832]
['SOLEYHO12AB0188A85' 'Got My Mojo Workin' 'ARAGB2O1187FB3A161' 0
 338.23302]
['SOFFKZS12AB017F194' 'A Higher Place (Album Version)'
 'ARBEBBY1187B9B43DB' 1994 236.17261]
['SOOJPRH12A8C141995' 'Loaded Like A Gun' 'ARBGXIG122988F409D' 0 173.19138]
['SOGVQGJ12AB017F169' 'Ten Tonne' 'AR62SOJ1187FB47BB5' 2005 337.68444]
['SOXILUQ12A58A7C72A' 'Jenny Take a Ride' 'ARP6N5A1187B99D1A3' 2004
 207.43791]
['SOFNOQK12AB01840FC' 'Kutt Free (DJ Volume Remix)' 'ARNNKDK1187B98BBD5' 0
 407.37914]
['SOBCOSW12A8C13D398' 'Rumba De Barcelona' 'AR7SMBG1187B9B9066' 0
 218.38322]
['SOBAYLL12A8C138AF9' 'Sono andati? Fingevo di dormire'
 'ARDR4AC1187FB371A1' 0 511.16363]
['SOBKWDJ12A8C13B2F3' 'Wild Rose (Back 2 Basics Mix)' 'AR36F9J1187FB406F1'
 0 230.71302]
['

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [9]:
for artist in artist_data:
    cur.execute(artist_table_insert, artist)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [19]:
log_files = get_files("data/log_data")

In [20]:
filepath = log_files[1]

In [12]:
df = pd.read_json(filepath, lines = True)

df.head()
dt = df[df['page'] == 'NextSong']
#print(len(dt))
dt.values
t = dt['ts']
#print(dt)
date = pd.to_datetime(t, unit = 'ms')
print(date)
date_data = pd.DataFrame({'start_time':date.dt.time,'hour':date.dt.hour,'day':date.dt.day,'week':date.dt.week,'month':date.dt.month,'year':date.dt.year,'weekday':date.dt.weekday})
print(len(date_data.values))
#date
#t.values
#print(df.values)
#dt = []
#for i in range (len(log_files)):
    
    

0    2018-11-11 02:33:56.796
1    2018-11-11 02:36:10.796
2    2018-11-11 02:40:34.796
4    2018-11-11 04:36:13.796
5    2018-11-11 04:36:46.796
6    2018-11-11 09:56:28.796
7    2018-11-11 10:00:47.796
8    2018-11-11 10:04:07.796
9    2018-11-11 10:08:28.796
10   2018-11-11 10:12:00.796
12   2018-11-11 10:27:43.796
13   2018-11-11 10:33:44.796
14   2018-11-11 10:36:22.796
15   2018-11-11 10:40:14.796
16   2018-11-11 10:43:23.796
17   2018-11-11 10:46:18.796
23   2018-11-11 11:14:51.796
25   2018-11-11 13:41:22.796
27   2018-11-11 13:52:23.796
28   2018-11-11 13:58:22.796
30   2018-11-11 14:05:43.796
31   2018-11-11 14:07:54.796
32   2018-11-11 14:09:36.796
33   2018-11-11 14:14:29.796
35   2018-11-11 14:15:06.796
36   2018-11-11 14:17:52.796
37   2018-11-11 14:19:37.796
38   2018-11-11 14:24:58.796
39   2018-11-11 14:30:05.796
40   2018-11-11 14:33:53.796
               ...          
57   2018-11-11 15:10:12.796
58   2018-11-11 15:14:56.796
59   2018-11-11 16:01:19.796
60   2018-11-1

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [27]:
df = pd.DataFrame(columns = pd.read_json(log_files[0],lines = True).columns)
#date_df = pd.DataFrame(columns = ['start_time','hour','day','week','month','year','weekday'])
s = 0
for file in log_files:    
    data = pd.read_json(file, lines = True)
    s += len(data)
    df = df.append(data)
dt = df[df['page'] == 'NextSong']
t = dt['ts']
date = pd.to_datetime(t, unit = 'ms')
date_df = pd.DataFrame({'start_time':date,'hour':date.dt.hour,'day':date.dt.day,'week':date.dt.week,'month':date.dt.month,'year':date.dt.year,'weekday':date.dt.weekday})
user_df = pd.DataFrame({'user_id':df['userId'],'first_name':df['firstName'],'last_name':df['lastName'],'gender':df['gender'],'level':df['level']})

print('data entry: {}, date_df: {}, user_df: {}'.format(s,len(date_df),len(user_df)))
print(user_df.values)

data entry: 8056, date_df: 6820, user_df: 8056
[['53' 'Celeste' 'Williams' 'F' 'free']
 ['10' 'Sylvie' 'Cruz' 'F' 'free']
 ['53' 'Celeste' 'Williams' 'F' 'free']
 ...
 ['49' 'Chloe' 'Cuevas' 'F' 'free']
 ['6' 'Cecilia' 'Owens' 'F' 'free']
 ['6' 'Cecilia' 'Owens' 'F' 'free']]


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [14]:
for i, row in time_df.iterrows():
    
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [15]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [1]:
for index, row in df.iterrows():
    print('{},{},{}'.format(row.song, row.artist, row.length))
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    print(results)
    print('{},{},{},{},{},{}'.format(pd.to_datetime(row.ts,unit = 'ms'), row.userId, row.level, songid, artistid, row.sessionId))
    songplay_data = (pd.to_datetime(row.ts,unit = 'ms'), row.userId, row.level, songid, artistid, row.sessionId)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

NameError: name 'df' is not defined

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [1]:
conn.close()

NameError: name 'conn' is not defined

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.